In [1]:
# Use autoplot 207's code for this app
import sys
import datetime
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from shapely.geometry import Point
from pyiem.plot import MapPlot, nwssnow
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
STORM_NUMBER = 17
TITLE = "9 February 2023"
SUBTITLE = "8 AM 10 February 2023"
SETPOINT_LOCS = {}
sts = utc(2023, 2, 9, 18)
ets = utc(2023, 2, 10, 20)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
        """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
        conn,
        params=(sts - datetime.timedelta(days=1), ets)
        )
    print(df)
    mp.plot_values(df.lon.values, df.lat.values, df.magnitude.values, fmt="%.2f", labelbuffer=1, color='purple')

def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2022-2023 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1])
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [16]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
    ]
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
        ('WLBI4', 2),
        ('IA-WS-8', 2),
        (3, 10),
        (8, 10),
        (9, 10),

    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=4, upper=8)
    # draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename='230210.png')
    mp.close()

main()

     state  wfo   val      lon      lat                              geo  \
0       IA  DVN   9.4 -90.7300  42.4100   POINT (759234.612 -244937.769)   
1       WI  ARX   9.0 -90.6000  42.5800   POINT (767705.556 -224912.962)   
2       WI  MKX   8.0 -89.5100  43.2500   POINT (846781.533 -140037.584)   
3       WI  ARX  10.0 -90.4300  42.5100   POINT (782399.995 -231046.106)   
4       WI  MKX   8.0 -89.5300  43.3200   POINT (844196.337 -132508.871)   
...    ...  ...   ...      ...      ...                              ...   
1277    ME  GYX   0.0 -70.6350  43.2356   POINT (2314991.000 229395.790)   
1280    MI  DTX   0.0 -83.6933  42.2416  POINT (1331726.883 -173404.107)   
1281    MI  DTX   0.0 -83.7167  42.3000  POINT (1328593.076 -167384.852)   
1282    MI  DTX   0.0 -83.5919  43.0675   POINT (1321901.480 -81249.917)   
1283    MI  DTX   0.0 -83.4000  43.4500   POINT (1328608.672 -36222.674)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
0                  True  

/opt/miniconda3/envs/prod/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
